In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U accelerate
!pip install -q -U transformers
!pip install huggingface_hub --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
model_id = "google/gemma-2b"
model = AutoModelForCausalLM.from_pretrained(model_id).to("cpu")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): GemmaRMSNorm((2048,), 

## Prepare Dataset

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import concatenate_datasets

train_dataset_path = '/content/train_dataset.csv'
dataset = load_dataset('csv', data_files=train_dataset_path)

In [ ]:
def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, (context info), and answer.

    :param data_point: dict: Data point
    :return: dict: Data point with the added "prompt" field
    """
    prompt_template = """
    <start_of_turn>user
    You are an intelligent AI specialized in generating SQL queries.
    Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
    Please provide the SQL query corresponding to the given prompt and context:

    Prompt:
    convert a PostgreSQL query into ClickHouse dialect using PG schema, clickhouse schema and PG query

    Context:
    {sql_context}<end_of_turn>
    <start_of_turn>model
    {sql}<end_of_turn>
    """
    sql_context = """
    Natural languge of query: {natural_languge},
    PostgreSQL query: {pg_query},
    PostgreSQL Schema: {pg_db_schema},
    ClickHouse Schema: {ch_db_schema}
    """.format(
        natural_languge=data_point["natural_languge"],
        pg_query=data_point["pg_query"],
        pg_db_schema=data_point["pg_db_schema"],
        ch_db_schema=data_point["ch_db_schema"],
    )
    prompt_text = prompt_template.format(sql_context=sql_context, sql=data_point["ch_query"])
    data_point["prompt"] = prompt_text

    return data_point

dataset = dataset.map(generate_prompt)

In [ ]:
print(dataset['train']['prompt'][99])


    <start_of_turn>user
    You are an intelligent AI specialized in generating SQL queries.
    Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
    Please provide the SQL query corresponding to the given prompt and context:

    Prompt:
    convert a PostgreSQL query into ClickHouse dialect using PG schema, clickhouse schema and PG query

    Context:
    
    Natural languge of query: What college had a LB in the draft?,
    PostgreSQL query: sql SELECT College FROM table_name WHERE Position = 'LB';,
    PostgreSQL Schema: sql CREATE TABLE table_name ( College VARCHAR(255), Position VARCHAR(50) );,
    ClickHouse Schema: sql CREATE TABLE table ( College String, Position String ) ENGINE = MergeTree ORDER BY tuple();
    <end_of_turn>
    <start_of_turn>model
    sql SELECT College FROM table WHERE Position = 'LB'<end_of_turn>
    


In [ ]:
dataset = dataset.shuffle(seed=2000)
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

## LORA learning

In [ ]:
# Функция поискать только линейных слоев
def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["layers.0.self_attn.q_proj"],
    lora_dropout=0.05,
    bias="none",
)

In [ ]:
from peft import PeftModel, LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    fp16=False,
    logging_dir="./logs",
    learning_rate=2e-4,
    weight_decay=0.01,
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1595: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


trainable params: 32,768 || all params: 2,506,205,184 || trainable%: 0.0013


No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
new_model = "pretrained_model"
trainer.train()
trainer.model.save_pretrained(new_model)

Step,Training Loss
500,1.547400


In [ ]:
model.push_to_hub(commit_message="Initial commit of finetuned model")